<a href="https://colab.research.google.com/github/seoyen1122/solar_rag/blob/main/mmlu_pro/law_%EB%91%90%EB%B2%88%EC%A7%B8%EC%B6%94%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 103.6 MB/s eta 0:00:00


In [ ]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.8/473.8 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 100.9 MB/s eta 0:00:00


In [ ]:
import glob
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_upstage import UpstageEmbeddings, ChatUpstage
import pdfplumber
import re
import json

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/2025_2/nlp

/content/drive/MyDrive/Colab Notebooks/2025_2/nlp


In [ ]:
INDEX_NAME = "faiss_index_law"
HELIX_PDFS = [
    ("helix_torts_mini-outline.pdf", "torts"),
    ("helix_real_property_mini-outline.pdf", "real property"),
    ("helix_evidence_mini-outline.pdf", "evidence"),
    ("helix_criminal_procedure_mini-outline.pdf", "criminal procedure"),
    ("helix_criminal_law_mini-outline.pdf", "criminal law"),
    ("helix_contracts_mini-outline.pdf", "contracts"),
    ("helix_constitutional_law_mini-outline.pdf", "constitutional law"),
    ("helix_civil_procedure_mini-outline.pdf", "civil procedure")
]

In [ ]:
OUTPUT_JSONL = "helix_structured_all.jsonl"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 100

In [ ]:
UPSTAGE_API_KEY = "up_g7T2cQoLKZH6Oi2n4MHOW706XAdSs"
embeddings = UpstageEmbeddings(api_key = UPSTAGE_API_KEY, model="solar-embedding-1-large")

In [ ]:
db = FAISS.load_local(
    INDEX_NAME,
    embeddings,
    allow_dangerous_deserialization=True,  # langchain>=0.2.x에서 필요
)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
)

In [ ]:
def is_heading(line: str, subject: str | None = None) -> bool:
    """Helix 스타일에 맞춘 제목 감지 휴리스틱."""
    line = line.strip()
    if not line:
        return False
    # 페이지 번호, 푸터 등 버리기
    if re.match(r"^\d+\s+Helix Bar Review", line):
        return False

    if "Helix Bar Review" in line:
        return False

    if subject and line.upper() == subject.upper():
        return False

    if line.startswith("Prima Facie Case:"):
        return False

    # 완전 대문자 + 숫자/공백/기호 정도면 heading 가능성 높음
    if re.match(r"^[A-Z0-9\s\-’'&/]+$", line) and 5 <= len(line) <= 50:
        return True

    # 맨 앞 대문자, 나머지 소문자/공백 정도의 짧은 라인 (예: "Defenses to Intentional Torts")
    if len(line) <= 60 and line[0].isupper() and line.isprintable() and line.count(".") == 0:
        # 문장 느낌보다는 제목 느낌
        return True

    return False


In [ ]:
def extract_lines_from_pdf(pdf_path: str):
    lines = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""
            # 줄 단위 split
            for line in text.splitlines():
                line = line.rstrip()
                if line:
                    lines.append(line)
    return lines

In [ ]:
def build_chunk_list_from_lines(lines, subject):
    """Helix outline 텍스트 라인들을 section_title 기반 chunk_list로 변환."""
    chunk_list = []

    current_title = None
    current_text_lines = []

    for line in lines:
        if is_heading(line, subject):
            # INTRODUCTION 같은 pure 안내 섹션은 건너뛰고 싶으면 여기서 필터
            if current_title and current_text_lines:
                chunk_list.append({
                    "section_title": current_title,
                    "text": "\n".join(current_text_lines).strip()
                })
            current_title = line.strip()
            current_text_lines = []
        else:
            # 본문 라인
            if current_title is None:
                # 아직 제목이 안나온 상태면 스킵하거나 "INTRO"로 묶을 수 있음
                continue
            current_text_lines.append(line.strip())

    # 마지막 섹션 flush
    if current_title and current_text_lines:
        chunk_list.append({
            "section_title": current_title,
            "text": "\n".join(current_text_lines).strip()
        })

    return chunk_list

In [ ]:
def main():
    with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
        for pdf_path, subject in HELIX_PDFS:
            lines = extract_lines_from_pdf(pdf_path)
            chunk_list = build_chunk_list_from_lines(lines, subject)

            # 제목은 대충 파일명에서 생성
            title = f"Helix {subject.title()} Mini-Outline"

            meta = {
                "source_url": f"helix::{pdf_path}",
                "title": title,
                "doc_type": "helix",
                "subject": subject,
                "chunk_list": chunk_list,
            }

            f.write(json.dumps(meta, ensure_ascii=False) + "\n")

    print(f"[DONE] Written structured Helix jsonl to {OUTPUT_JSONL}")


In [ ]:
if __name__ == "__main__":
    main()

[DONE] Written structured Helix jsonl to helix_structured_all.jsonl


jsonl to index.faiss & index.pkl

In [ ]:
import json
from langchain_core.documents import Document

# 네가 이미 쓰고 있던 함수 그대로 사용
def chunk_text(text, max_chars=800, overlap_chars=200):
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunks.append(chunk)
        if end == n:
            break
        start = end - overlap_chars
    return chunks

def load_chunks_multi(jsonl_paths, max_chars=None, overlap_chars=200):
    chunks = []
    metadata = []

    for jsonl_path in jsonl_paths:
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                rec = json.loads(line)
                source_url = rec["source_url"]
                title = rec["title"]
                doc_type = rec.get("doc_type", "wex")
                subject = rec.get("subject")   # helix용
                chunk_list = rec.get("chunk_list", [])

                for sec_idx, c in enumerate(chunk_list):
                    section_title = c.get("section_title", title)
                    text = c.get("text", "")
                    if not text.strip():
                        continue

                    base_meta = {
                        "source_url": source_url,
                        "title": title,
                        "doc_type": doc_type,
                        "section_title": section_title,
                        "section_index": sec_idx,
                    }
                    if subject is not None:
                        base_meta["subject"] = subject

                    if max_chars is None:
                        chunks.append(text)
                        m = base_meta.copy()
                        m["subchunk_index"] = 0
                        metadata.append(m)
                    else:
                        for sub_idx, ch in enumerate(
                            chunk_text(text, max_chars, overlap_chars)
                        ):
                            chunks.append(ch)
                            m = base_meta.copy()
                            m["subchunk_index"] = sub_idx
                            metadata.append(m)

    return chunks, metadata

# ★ 헬릭스 jsonl만 로드
helix_chunks, helix_meta = load_chunks_multi(
    ["helix_structured_all.jsonl"],
    max_chars=800,
    overlap_chars=200,
)

helix_docs = [
    Document(page_content=txt, metadata=m)
    for txt, m in zip(helix_chunks, helix_meta)
]

print("num helix docs:", len(helix_docs))


num helix docs: 629


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings

INDEX_NAME = "faiss_index_law"  # index.faiss / index.pkl 있는 폴더

# 1) 임베딩 & 기존 인덱스 로드
embeddings = UpstageEmbeddings(api_key = UPSTAGE_API_KEY, model="solar-embedding-1-large")
db = FAISS.load_local(
    INDEX_NAME,
    embeddings,
    allow_dangerous_deserialization=True,  # langchain 0.2+ 이면 필요
)
print("loaded existing index")

# 2) 헬릭스 문서 추가
db.add_documents(helix_docs)
print("added helix docs")

# 3) 다시 저장 (덮어쓰기)
db.save_local(INDEX_NAME)
print(f"updated index saved to folder: {INDEX_NAME}")


loaded existing index
added helix docs
updated index saved to folder: faiss_index_law
